# How to get MOC data from VESPA in Python

Illustrate search for CDPP TAP MOC for vespa using 'pyvo' and 'astropy'

## Import required modules

In [13]:
import pyvo as vo
from mpl_toolkits.basemap import Basemap
import re

## Search a specific TAP service

We are looking for VVEx - VIRTIS/Venus Express database since it has a MOC

### Search the database containing this service

In [14]:
nameServiceSearched = "VVEx"

In [15]:
databasesList = vo.registry.search(servicetype='tap',keywords=[nameServiceSearched], includeaux=True)
print(databasesList.to_table)

<bound method DALResults.to_table of <Table length=1>
           ivoid                  res_type     ...  intf_types  intf_roles
                                               ...                        
           object                  object      ...    object      object  
---------------------------- ----------------- ... ------------ ----------
ivo://padc.obspm.planeto/tap vs:catalogservice ... vs:paramhttp        std>


#### Get the first result and print its url

In [16]:
database = databasesList[0] # First result
print("Url: ",database.access_url) # Url used to access the service

Url:  http://voparis-tap-planeto.obspm.fr/tap


### Search the service in the database that was found

table_limit is too low by default so it needs to be specified here

In [19]:
for s in list(database.get_tables(table_limit=50).keys()) :
    if re.search("^.*"+nameServiceSearched.upper()+".*$", str(s).upper()) :
        service = database.get_tables(table_limit=50)[s]
        break

In [20]:
print("Service: ",service.name)

Service:  vvex.epn_core


## Check if a a table has a coverage key (and so maybe a MOC)

In [21]:
def doContainMOC(table) :
    if table == None :
        return False
    for field in table.columns :
        if re.search("^.*<BaseParam name=\"coverage\"/>.*$", str(field)) :
            return True
    return False

In [22]:
#if doContainMOC(service) :
    # queries to get MOC data